## Multi-Armed Bandit Problem

We will show multiple methods to solve this partial-information sequential decision making problem, referred to as the Multi Armed Bandit Problem.

In [1]:
# Importing the libraries
import numpy as np
import sys

import matplotlib.pyplot as plt
import pandas as pd

#np.set_printoptions(2)
np.set_printoptions(3,suppress=True)
np.random.seed(67)

### Epsilon-greedy Agent method

In [2]:
ground_truth_prob = np.random.rand(10)
number_of_levers_count = np.zeros(10)

agents_prob = np.zeros(10)
reward_count = np.zeros(10)

In [3]:
num_episode = 4000
e = 0.33

In [4]:
for _ in range(num_episode):
    
    # Either choose a greedy action or explore
    if e > np.random.uniform():
        which_lever_to_pull = np.random.randint(0,10)
    else:
        which_lever_to_pull = np.argmax(agents_prob)

    # now pull the lever 
    if ground_truth_prob[which_lever_to_pull] > np.random.uniform():
        reward = 1
    else:
        reward = 0

    # now update the lever count and expected value
    number_of_levers_count[which_lever_to_pull] += 1
    reward_count[which_lever_to_pull] = reward_count[which_lever_to_pull] \
                                        + reward
    agents_prob[which_lever_to_pull] = agents_prob[which_lever_to_pull] \
                    + (1/number_of_levers_count[which_lever_to_pull]) * \
                      (reward - agents_prob[which_lever_to_pull])

In [5]:
print('\n-------------------------')
print("Number of Levers Count Each: ", number_of_levers_count)
print("Sum of lever must match with # episode : ",
      number_of_levers_count.sum(), ' : ', num_episode)


-------------------------
Number of Levers Count Each:  [ 136. 2847.  121.  106.  118.  120.  149.  135.  123.  145.]
Sum of lever must match with # episode :  4000.0  :  4000


In [6]:
print('\n-------------------------')
print("Reward Over Time Tracking : ", reward_count)
print("Agent's weight : ", agents_prob)
print("Agent's Probability Guess : ", reward_count/number_of_levers_count )
print("Agents Guess Best Lever : ", 
      np.argmax(reward_count/number_of_levers_count))


-------------------------
Reward Over Time Tracking :  [ 111. 2803.   34.   38.   34.  101.   11.  113.   38.   42.]
Agent's weight :  [0.82 0.98 0.28 0.36 0.29 0.84 0.07 0.84 0.31 0.29]
Agent's Probability Guess :  [0.82 0.98 0.28 0.36 0.29 0.84 0.07 0.84 0.31 0.29]
Agents Guess Best Lever :  1


In [7]:
print('\n-------------------------')
print("Ground Truth Full list : ", ground_truth_prob)
print("Ground Truth Best Lever : ", np.argmax(ground_truth_prob))

# -- end code --


-------------------------
Ground Truth Full list :  [0.81 0.99 0.29 0.39 0.27 0.83 0.1  0.86 0.34 0.36]
Ground Truth Best Lever :  1


### Policy Gradient with log-likelihood 

We now are going to update the agent’s weight. 

In [9]:
ground_truth_prob = np.random.rand(10)
number_of_levers_count = np.zeros(10)
agents_prob  = np.zeros(10)
reward_count = np.zeros(10)

In [10]:
num_episode = 10000
e = 0.99
learning_rate = 0.5   

In [11]:
for _ in range(num_episode):
    
    # Either choose a greedy action or explore
    if e > np.random.uniform():
        which_lever_to_pull = np.random.randint(0,10)
    else:
        which_lever_to_pull = np.argmax(agents_prob)

    # now pull the lever 
    if ground_truth_prob[which_lever_to_pull] > np.random.uniform():
        reward = 1
    else:
        reward = 0

    # now update the lever count and expected value
    number_of_levers_count[which_lever_to_pull] += 1
    loss =  - np.log( agents_prob[which_lever_to_pull] + 1e-3 ) * reward 
    agents_prob[which_lever_to_pull] = agents_prob[which_lever_to_pull] \
    - learning_rate * (-1/(agents_prob[which_lever_to_pull]+1e-3)) * \
      reward
    reward_count[which_lever_to_pull] = reward_count[which_lever_to_pull] \
                                        + reward
    e = e * 0.9999

In [12]:
print('\n-------------------------')
print("Number of Levers Count Each: ", number_of_levers_count)
print("Sum of lever must match with # episode : ",number_of_levers_count.sum(), ' : ',
      num_episode)


-------------------------
Number of Levers Count Each:  [ 653.  635.  618.  633.  647.  591.  650. 4363.  615.  595.]
Sum of lever must match with # episode :  10000.0  :  10000


In [13]:
print('\n-------------------------')
print("Reward Over Time Tracking : ", reward_count) 
print("Agent's weight : ", agents_prob)
print("Agent's Probability Guess : ", reward_count/number_of_levers_count )
print("Agents Guess Best Lever : ", 
      np.argmax(reward_count/number_of_levers_count))


-------------------------
Reward Over Time Tracking :  [ 361.  534.  449.  215.   46.  223.  133. 4060.  451.   24.]
Agent's weight :  [500.36  500.533 500.448 500.214 500.045 500.222 500.132 504.043 500.45
 500.023]
Agent's Probability Guess :  [0.553 0.841 0.727 0.34  0.071 0.377 0.205 0.931 0.733 0.04 ]
Agents Guess Best Lever :  7


In [14]:
print('\n-------------------------')
print("Probability of each lever Ground Truth Full list : ", 
      ground_truth_prob)
print("Ground Truth Best Lever : ", np.argmax(ground_truth_prob))

# -- end code --


-------------------------
Probability of each lever Ground Truth Full list :  [0.546 0.859 0.686 0.332 0.06  0.386 0.213 0.933 0.723 0.047]
Ground Truth Best Lever :  7


### Policy Gradient with Multi Layer Perceptron Network

we can create two fully connected network, and using this method we can expand the Log-Likelihood method, rather than directly getting which bandit we want to pull, we can train a neural network that computes the optimal bandit by using back propagation.

In [15]:
def log_sig(x): return 1 / (1 + np.exp(-x))
def d_log_sig(x): return log_sig(x) * ( 1 - log_sig(x))

In [16]:
ground_truth_prob = np.random.rand(10)
number_of_levers_count = np.zeros(10)

agents_prob  = np.zeros(10)
reward_count = np.zeros(10)

In [17]:
num_episode = 20000
e = 0.1
learning_rate = 0.00000000001

In [18]:
class FNN():
    
    def __init__(self,inc,outc):
        self.w = np.random.randn(inc,outc) 

    def feed(self, input):
        self.input = input
        self.layer = np.dot(self.input,self.w)
        self.layerA = log_sig(self.layer)
        return self.layerA
    
    def back(self, grad):
        grad_part_1 = grad
        grad_part_2 = d_log_sig(self.layer)
        grad_part_3 = self.input

        grad_middle = grad_part_1 * grad_part_2
        grad = grad_part_3.T.dot(grad_middle)
        grad_pass = grad_middle.dot(self.w.T)
        self.w = self.w - learning_rate * grad
        return grad_pass

In [19]:
l1 = FNN(10,30)
l2 = FNN(30,10)

In [20]:
for _ in range(num_episode):
    
    # Either choose a greedy action or explore
    if e > np.random.uniform():
        which_lever_to_pull = np.random.randint(0,10)
    else:
        which_lever_to_pull = np.argmax(agents_prob)

    temp = np.zeros((1,10))
    temp[0,which_lever_to_pull] = 1
    layer1 = l1.feed(temp)
    layer2 = l2.feed(layer1)
    which_lever_to_pull2 = np.argmax(layer2)

    # now pull the lever 
    if ground_truth_prob[which_lever_to_pull2] > np.random.uniform():
        reward = 1
    else:
        reward = 0

    # now update the lever count and expected value
    number_of_levers_count[which_lever_to_pull2] += 1
    loss =  - np.log( np.clip(agents_prob * temp,1e-10,1e10)  ) * reward 
    back_prop = (-1/ np.clip(agents_prob * temp,1e-10,1e10)  )* reward
    grad2 = l2.back(back_prop)
    grad1 = l1.back(grad2)
    agents_prob = agents_prob - learning_rate * grad1 
    reward_count[which_lever_to_pull2] = reward_count[which_lever_to_pull2]\
                                        + reward

In [22]:
print('\n-------------------------')
print("Number of Levers Count Each: ", number_of_levers_count)
print("Sum of lever must match with # episode : ",number_of_levers_count.sum(), ' : ',
      num_episode)


-------------------------
Number of Levers Count Each:  [   99.   254.     0. 17597.   213.     0.   183.  1247.   191.   216.]
Sum of lever must match with # episode :  20000.0  :  20000


In [23]:
print('\n-------------------------')
print("Reward Over Time Tracking : ", reward_count)
print("Agent's weight : ", agents_prob)
print("Agent's Probability Guess : ", 
      reward_count/(number_of_levers_count+1e-10))
print("Agents Guess Best Lever : ", 
      np.argmax(reward_count/(number_of_levers_count+1e-10)))


-------------------------
Reward Over Time Tracking :  [  33.   76.    0. 1700.   52.    0.   44.  123.   23.  164.]
Agent's weight :  [[ 4.587  3.857  3.427  3.64  -2.069  1.604  4.651  5.307  0.077  7.685]]
Agent's Probability Guess :  [0.333 0.299 0.    0.097 0.244 0.    0.24  0.099 0.12  0.759]
Agents Guess Best Lever :  9


In [24]:
print('\n-------------------------')
print("Probability of each lever Ground Truth Full list : ", 
      ground_truth_prob)
print("Ground Truth Best Lever : ", np.argmax(ground_truth_prob))

# -- end code --


-------------------------
Probability of each lever Ground Truth Full list :  [0.398 0.264 0.578 0.096 0.212 0.957 0.235 0.1   0.144 0.711]
Ground Truth Best Lever :  5


### Linear Reward Inaction

This method is simpler than using a neural network, we are simply going to increase the probability of getting the optimal bandit to be choose in the given probability distribution. We are only going to update when we receive a reward from the bandit. And when we do not have any reward we aren’t going to update anything.

In [27]:
ground_truth_prob = np.random.rand(10)
number_of_levers_count = np.zeros(10)

agents_prob = np.ones(10) * 0.1
reward_count = np.zeros(10)

In [28]:
num_episode = 40000
e = 0.33
alpha = 0.2

In [29]:
for _ in range(num_episode):
    
    # Either choose a greedy action or explore
    if e > np.random.uniform():
        which_lever_to_pull = np.random.randint(0,10)
    else:
        which_lever_to_pull = np.random.choice(10, p=agents_prob)

    # now pull the lever 
    if ground_truth_prob[which_lever_to_pull] > np.random.uniform():
        reward = 1
    else:
        reward = 0

    # now update the lever count and expected value
    number_of_levers_count[which_lever_to_pull] += 1
    reward_count[which_lever_to_pull] = reward_count[which_lever_to_pull] \ 
                                        + reward
    if reward == 1:
        mask1 = np.zeros(10)
        mask2 = np.ones(10)
        mask1[which_lever_to_pull] = 1
        mask2[which_lever_to_pull] = 0
        lever_won_vector = (agents_prob + alpha * (1- agents_prob)) \
                            * mask1
        rest_of_levers   = ((1-alpha) * agents_prob) * mask2
        agents_prob = lever_won_vector + rest_of_levers

In [30]:
print('\n-------------------------')
print("Number of Levers Count Each: ", number_of_levers_count)
print("Sum of lever must match with # episode : ",
      number_of_levers_count.sum(),' : ', num_episode)


-------------------------
Number of Levers Count Each:  [2120. 3006. 4534. 4480. 3147. 6465. 2886. 7687. 2558. 3117.]
Sum of lever must match with # episode :  40000.0  :  40000


In [31]:
print('\n-------------------------')
print("Reward Over Time Tracking : ", reward_count)
print("Agent's weight : ", agents_prob)
print("Agent's Probability Guess : ", reward_count/number_of_levers_count )
print("Agents Guess Best Lever : ", np.argmax(reward_count/number_of_levers_count))


-------------------------
Reward Over Time Tracking :  [ 867. 1726. 3366. 3339. 1919. 5667. 1628. 7267. 1331. 1733.]
Agent's weight :  [0.264 0.371 0.008 0.168 0.    0.182 0.    0.008 0.    0.   ]
Agent's Probability Guess :  [0.409 0.574 0.742 0.745 0.61  0.877 0.564 0.945 0.52  0.556]
Agents Guess Best Lever :  7


In [32]:
print('\n-------------------------')
print("Ground Truth Full list : ", ground_truth_prob)
print("Ground Truth Best Lever : ", np.argmax(ground_truth_prob))

# -- end code --


-------------------------
Ground Truth Full list :  [0.412 0.589 0.742 0.736 0.606 0.878 0.56  0.944 0.513 0.555]
Ground Truth Best Lever :  7


### Linear Reward Penalty

W simply expand the Linear Reward Inaction method, as seen above, when we do not get a reward for certain bandit. We are simply going to decrease the probability of that bandit being chosen in the next round.

In [33]:
ground_truth_prob = np.random.rand(10)
number_of_levers_count = np.zeros(10)
agents_prob = np.ones(10) * 0.1
reward_count = np.zeros(10)

In [34]:
num_episode = 40000
e = 0.33
alpha = 0.2
beta = 0.8 

In [35]:
for _ in range(num_episode):
    
    # Either choose a greedy action or explore
    if e > np.random.uniform():
        which_lever_to_pull = np.random.randint(0,10)
    else:
        which_lever_to_pull = np.random.choice(10, p=agents_prob)

    # now pull the lever 
    if ground_truth_prob[which_lever_to_pull] > np.random.uniform():
        reward = 1
    else:
        reward = 0

    # now update the lever count and expected value
    number_of_levers_count[which_lever_to_pull] += 1
    reward_count[which_lever_to_pull] = reward_count[which_lever_to_pull] \
                                        + reward
    mask1 = np.zeros(10)
    mask2 = np.ones(10)
    mask1[which_lever_to_pull] = 1
    mask2[which_lever_to_pull] = 0
    if reward == 1:
        lever_won_vector = (agents_prob + alpha * (1- agents_prob)) \
                           * mask1
        rest_of_levers   = ((1-alpha) * agents_prob) * mask2
        agents_prob = lever_won_vector + rest_of_levers
    else:
        rest_of_levers = ( beta / (len(ground_truth_prob)-1) \
                         + (1-beta) *agents_prob ) * mask2
        lever_lose_vector = ( (1-beta)*agents_prob )* mask1
        agents_prob = lever_lose_vector + rest_of_levers

In [36]:
print('\n-------------------------')
print("Number of Levers Count Each: ", number_of_levers_count)
print("Sum of lever must match with # episode : ",number_of_levers_count.sum(), ' : ', num_episode)


-------------------------
Number of Levers Count Each:  [4002. 4946. 4369. 4667. 3397. 3952. 4274. 4038. 3247. 3108.]
Sum of lever must match with # episode :  40000.0  :  40000


In [37]:
print('\n-------------------------')
print("Reward Over Time Tracking : ", reward_count)
print("Agent's weight : ", agents_prob)
print("Agent's Probability Guess : ", reward_count/number_of_levers_count )
print("Agents Guess Best Lever : ", np.argmax(reward_count/number_of_levers_count))


-------------------------
Reward Over Time Tracking :  [3143. 4866. 3981. 4545. 1776. 3204. 3473. 3076. 1758. 1365.]
Agent's weight :  [0.009 0.01  0.01  0.137 0.073 0.002 0.009 0.53  0.21  0.009]
Agent's Probability Guess :  [0.785 0.984 0.911 0.974 0.523 0.811 0.813 0.762 0.541 0.439]
Agents Guess Best Lever :  1


In [38]:
print('\n-------------------------')
print("Ground Truth Full list : ", ground_truth_prob)
print("Ground Truth Best Lever : ", np.argmax(ground_truth_prob))

# -- end code --


-------------------------
Ground Truth Full list :  [0.793 0.984 0.907 0.973 0.531 0.823 0.816 0.751 0.547 0.436]
Ground Truth Best Lever :  1


## Ad CTR Optimization using Multi-armed Bandit Aproach

Suppose an advertising company is running 10 different ads targeted towards a similar set of population on a webpage. We have a 1 if the ad was clicked by a user, and 0 if it was not, for each Ad.

Obtained data from https://drive.google.com/file/d/1whkIInL4FKeHg2IfdcbT1j18L26fg9aF/view

In [3]:
# Random Selection
# Importing the dataset
dataset = pd.read_csv('/Users/chrisjcc/Downloads/Ads_Optimisation.csv')

In [4]:
# Implementing Random Selection
import random
N = 10000
d = 10

In [5]:
ads_selected = []
total_reward = 0

In [6]:
for n in range(0, N):
    ad = random.randrange(d)
    ads_selected.append(ad)
    reward = dataset.values[n, ad]
    total_reward = total_reward + reward

In [7]:
pd.Series(ads_selected).tail(1000).value_counts(normalize=True)

7    0.118
8    0.111
6    0.106
0    0.104
2    0.102
9    0.099
3    0.099
4    0.094
1    0.086
5    0.081
dtype: float64

Total reward for the random selection algorithm comes out to be 1170. As this algorithm is not learning anything, it will not smartly select any ad which is giving the maximum return. And hence even if we look at the last 1000 trials, it is not able to find the optimal ad.

In [8]:
# Implementing Upper Confidence Bound (UCB)
import math
N = 10000
d = 10

In [9]:
ads_selected = []
numbers_of_selections = [0] * d
sums_of_reward = [0] * d
total_reward = 0

In [10]:
for n in range(0, N):
    ad = 0
    max_upper_bound = 0
    for i in range(0, d):
        if (numbers_of_selections[i] > 0):
            average_reward = sums_of_reward[i] / numbers_of_selections[i]
            delta_i = math.sqrt(2 * math.log(n+1) / numbers_of_selections[i])
            upper_bound = average_reward + delta_i
        else:
            upper_bound = 1e400
        if upper_bound > max_upper_bound:
            max_upper_bound = upper_bound
            ad = i
    ads_selected.append(ad)
    numbers_of_selections[ad] += 1
    reward = dataset.values[n, ad]
    sums_of_reward[ad] += reward
    total_reward += reward

In [11]:
pd.Series(ads_selected).head(1000).value_counts(normalize=True)

7    0.192
4    0.153
0    0.115
8    0.096
3    0.093
6    0.080
1    0.080
2    0.077
5    0.059
9    0.055
dtype: float64

The total_reward for UCB comes out to be 2125. Clearly, this is much better than random selection and indeed a smart exploration technique that can significantly improve our strategy to solve a MABP.

After just 1500 trials, UCB is already favouring Ad #5 (index 4) which happens to be the optimal ad, and gets the maximum return for the given problem.